# Análisis cuantitativo del sistema inmunitario humano (interactivo)
## Efecto de un aumento del 20% en la masa del tejido adiposo

**Autores:** Raúl Mendoza y Adrián Ojeda
**Asignatura:** Bioinformática  

Este cuaderno analiza los datos suplementarios de Ron Milo et al. (PNAS, 2023) para:
1. describir la distribución basal de células inmunitarias por tejido,
2. visualizarla de forma interactiva,
3. simular un escenario de +20% de masa de tejido adiposo,
4. cuantificar el impacto sobre el total de células y masa inmunitaria,
5. interpretar los resultados y sus limitaciones.

### Pregunta de investigación
> **¿Hasta qué punto el aumento de la masa del tejido adiposo, como modelo simple de obesidad, modifica el reparto cuantitativo de las células inmunitarias en el organismo?**


In [93]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

def mostrar(fig):
    try:
        fig.show()
    except Exception:
        html = fig.to_html(include_plotlyjs="cdn")
        display(HTML(html))

SD01 = "pnas.2308511120.sd01.xlsx"
SD02 = "pnas.2308511120.sd02.xlsx"

assert os.path.exists(SD01), "No se encuentra pnas.2308511120.sd01.xlsx"
assert os.path.exists(SD02), "No se encuentra pnas.2308511120.sd02.xlsx"

sd01 = pd.ExcelFile(SD01)
sd02 = pd.ExcelFile(SD02)

tissues = sd01.parse("Tissues")
mass = sd01.parse("Mass")
cell_type_densities = sd02.parse("cell_type_densities")
total_immune_densities = sd02.parse("total_immune_densities")
numbers_by_cell_type = sd02.parse("numbers_by_cell_type")
mass_by_cell_type = sd02.parse("mass_by_cell_type")
totals = sd02.parse("totals")

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]
    df.columns = [str(c).strip() for c in df.columns]
    return df

tissues = clean_df(tissues)
mass = clean_df(mass)
cell_type_densities = clean_df(cell_type_densities)
total_immune_densities = clean_df(total_immune_densities)
numbers_by_cell_type = clean_df(numbers_by_cell_type)
mass_by_cell_type = clean_df(mass_by_cell_type)
totals = clean_df(totals)

tissues.head()


,tissue,subtissue,comment,mass,sem,groups,front line organ,for_final_table,child_10y,woman,man
0,Blood,NaN,NaN,5241.0,NaN,blood,NaN,Blood,2616.0,4251.0,5500.0
1,Red marrow,NaN,NaN,1500.0,NaN,marrow,NaN,Bone Marrow,630.0,900.0,1170.0
2,Lymph nodes,NaN,NaN,250.0,NaN,lymph system,NaN,Lymph system,99.0,185.0,225.0
3,Lymph vessels,NaN,NaN,450.0,NaN,connective,NaN,Lymph system,221.0,415.0,505.0
4,Spleen,NaN,NaN,150.0,20.0,lymph system,NaN,Lymph system,80.0,130.0,150.0


## 1. Descripción de los datos

- **Tissues**: masa de cada tejido en gramos (man = individuo de referencia).
- **total_immune_densities**: densidad total de células inmunitarias por tejido.
- **cell_type_densities**: densidades específicas de cada tipo celular en cada tejido.
- **numbers_by_cell_type**: número total de cada tipo de célula para el individuo de referencia.
- **mass_by_cell_type**: masa total de cada tipo de célula.

Con esto podemos pasar de: tejido → densidad → número → masa.


In [94]:
total_cells_ref = numbers_by_cell_type["tot_man"].sum()
total_mass_ref_g = mass_by_cell_type["mass"].sum()

avg_mass_per_cell = (
    mass_by_cell_type[["cell_type", "mass"]]
    .merge(numbers_by_cell_type[["cell_type", "tot_man"]], on="cell_type")
    .assign(g_per_cell=lambda d: d["mass"] / d["tot_man"])
    .set_index("cell_type")["g_per_cell"]
)

print(f"Total de células inmunitarias (referencia): {total_cells_ref:,.0f}")
print(f"Masa inmunitaria total (referencia): {total_mass_ref_g:,.2f} g")


Total de células inmunitarias (referencia): 1,837,950,387,444
Masa inmunitaria total (referencia): 1,224.49 g


Los valores que vemos junto encima son la línea base del individuo de referencia (73 kg). Toda la simulación posterior se comparará contra ellos para ver si el aumento del tejido adiposo cambia de verdad el “peso” del sistema inmunitario o si el efecto es más bien local.


In [95]:
top_density = (
    total_immune_densities[["tissue", "density"]]
    .sort_values("density", ascending=False)
    .head(15)
)

fig1 = px.bar(
    top_density,
    x="tissue",
    y="density",
    title="Gráfica 1. Tejidos con mayor densidad de células inmunitarias",
)
fig1.update_layout(xaxis_title="Tejido", yaxis_title="Células/g")
mostrar(fig1)



**Gráfica 1.** Los tejidos que aparecen arriba son los que concentran más células inmunitarias por gramo. Esto es clave para justificar qué tejidos son “inmunológicamente activos”: no basta con que un órgano sea grande, tiene que estar densamente poblado. Los tejidos linfoides deberían situarse aquí porque su función es precisamente esa.

Aquí vemos un patrón muy claro: hay cinco órganos que dominan la gráfica. Los tejidos linfoides—El Bazo y los Ganglios Linfáticos—lideran con casi 1.8 mil millones de células por gramo. Les siguen de cerca el Timo y las Amígdalas. Estos son los centros de comando del sistema inmunitario, y lo son por su extrema concentración celular.

A partir del quinto puesto, la densidad cae drásticamente. Órganos grandes como los Pulmones o el Colon son apenas visibles en esta escala. Esto valida un principio fundamental: La función se define por la densidad, no por el tamaño del órgano.

El Tejido Adiposo (TA) se comporta como los órganos de baja densidad. Nuestra simulación parte de la base de que, si bien aumentar la masa del TA en un 20% aumentará el número total de células inmunitarias en el organismo, no cambiará la jerarquía funcional. Los tejidos linfoides de alta densidad seguirán concentrando la actividad, mientras que el TA solo actuará como un gran reservorio de células, manteniendo su baja densidad intrínseca.

In [96]:

tissue_load = (
    tissues[["tissue", "man"]]
    .merge(total_immune_densities[["tissue", "density"]], on="tissue", how="inner")
    .assign(immune_load=lambda d: d["man"] * d["density"])
    .sort_values("immune_load", ascending=False)
)

fig2 = px.treemap(
    tissue_load,
    path=["tissue"],
    values="immune_load",
    title="Gráfica 2. Contribución aproximada de cada tejido a la carga inmunitaria total",
)
mostrar(fig2)


**Gráfica 2.** Acabamos de ver que la densidad es lo que define a un órgano como un centro de comando. Ahora, cambiamos el enfoque a la magnitud total. Un mapa de árbol, nos muestra el peso relativo de cada tejido en la carga inmunitaria total del cuerpo. Es decir, ¿Dónde reside la mayor cantidad de células, sin importar la concentración?

Noten cómo el panorama cambia radicalmente respecto a la Gráfica 1. Los tejidos que dominan aquí son aquellos que tienen un gran volumen, incluso si su densidad es media o baja. El líder indiscutible es la Médula Ósea Roja (Red Marrow), ocupando la mayor parte. 
- En la figura anterior, la Médula Roja era la quinta. Aquí es la primera, porque aunque no es el tejido más denso, su inmenso volumen como sitio de hematopoyesis le otorga el mayor número total de células.

Después de la Médula, encontramos a los Ganglios Linfáticos y el Bazo. Ellos sí mantienen un puesto alto porque tienen la doble ventaja: son densos y contribuyen con un volumen significativo. Pero el Bazo ahora comparte el escenario con tejidos que antes eran casi invisibles, como la Piel y los Pulmones.
- La Piel y los Pulmones aparecen grandes. No es que sean muy densos, sino que su gran área superficial y volumen total les permite albergar una enorme cantidad de células inmunitarias 'de primera línea' que, sumadas, representan una parte importante de la carga total.

En esta gráfica, vemos que responde a nuestro enfoque: ¿Qué tejidos aportan más al sistema inmune en total?. Y la respuesta es una combinación: La Médula Ósea: Como fábrica (Masa). Ganglios/Bazo: Como centros de procesamiento (Densidad y Masa). Piel/Pulmones: Como grandes defensas periféricas (Masa).

Al contrastar ambas figuras, tenemos nuestro marco de referencia: el sistema inmune tiene centros de control densos y grandes reservorios de volumen. Esto nos lleva directamente a nuestra simulación. Si el tejido adiposo (TA) aumenta, ¿a qué categoría afectará? ¿Se convertirá en un centro de control o simplemente en un reservorio más grande, como la Piel o los Pulmones? Los siguientes datos de la simulación nos darán la respuesta.

In [97]:
df_mass_density = (
    tissues[["tissue", "man"]]
    .merge(total_immune_densities[["tissue", "density"]], on="tissue", how="inner")
    .rename(columns={"man": "tissue_mass_g", "density": "immune_density_cells_g"})
)

fig3 = px.scatter(
    df_mass_density,
    x="tissue_mass_g",
    y="immune_density_cells_g",
    hover_name="tissue",
    title="Gráfica 3. Masa del tejido vs densidad inmunitaria (log-log)",
)
fig3.update_xaxes(type="log", title="Masa del tejido (g, log)")
fig3.update_yaxes(type="log", title="Densidad inmunitaria (células/g, log)")
mostrar(fig3)


**Gráfica 3.** Se aprecia que algunos tejidos son densos, activos y otros son voluminosos, contribuyen a la masa total. Esta gráfica nos da la imagen completa, cruzando ambas variables: Masa del Tejido, en el eje X contra Densidad Inmunitaria, en el eje Y. Ambos ejes están en escala logarítmica para abarcar todo el rango de datos.

Lo primero que observamos es una gran dispersión. No hay una línea clara que indique que los tejidos más grandes sean los más densos o viceversa.
- Enfatizar Dispersión: Si hubiera una correlación simple, veríamos los puntos alineados a lo largo de una curva o una línea recta, pero no es el caso. Esto demuestra que la actividad inmunológica no está predeterminada por el tamaño del órgano.

Se puede identificar cuatro grupos funcionales clave en este gráfico de dispersión, que refuerzan nuestras conclusiones previas:
- En la esquina superior izquierda se encuentran los puntos clave de la Figura 1, como los Ganglios, el Timo y las Amígdalas. Son tejidos pequeños en masa, pero con la máxima densidad. Son los centros de comando concentrados.
- En la esquina inferior derecha están los tejidos más voluminosos del cuerpo, como la Grasa (Tejido Adiposo), el Músculo Esquelético y el Tejido Conectivo. Son de gran masa, pero tienen la densidad más baja. Son los grandes reservorios o contenedores del cuerpo.
- El Bazo y la Médula Ósea se ubican en el centro-derecha. Ellos son los 'jugadores estrella': tienen ambos una alta densidad y contribuyen significativamente a la masa total.

Esta gráfica sienta las bases para nuestra simulación. Nuestro modelo simula el aumento del Tejido Adiposo, el cual se encuentra firmemente en el grupo Abajo y a la Derecha (baja densidad, alta masa).

Nuestro objetivo ahora es cuantificar: Si movemos el punto del Tejido Adiposo más a la derecha (aumentando su masa en un 20%), ¿cuál será el impacto en el Total de Células y la Masa Inmunitaria Total del individuo, y cómo se comparará ese cambio con los centros de control que están en la parte superior?

In [98]:
mass_sorted = mass_by_cell_type.sort_values("mass", ascending=False)
fig4 = px.sunburst(
    mass_sorted,
    path=["cell_type"],
    values="mass",
    title="Gráfica 4. Distribución de la masa inmunitaria por tipo celular",
)
mostrar(fig4)


**Gráfica 4.** Hasta ahora, hemos analizado dónde se localizan las células (tejido), pero ahora nos preguntamos: ¿Qué tipos de células son las más pesadas dentro del sistema inmunitario en su conjunto?" "Esta Gráfica 4 (un gráfico Sunburst) nos muestra la distribución de la masa inmunitaria total por tipo celular. Esto es clave, ya que para entender el impacto metabólico de la respuesta inmune, no basta con contar células, sino medir la masa que ocupan.

El resultado es contundente: los Macrófagos dominan absolutamente el gráfico. Ocupan la mayor porción de la masa inmunitaria total.
- Esto se debe a dos factores: Su gran tamaño celular en comparación con los linfocitos (como las células T y B) y su presencia ubicua en casi todos los tejidos del cuerpo, especialmente en el Tejido Adiposo, como adipocitos asociados a la grasa.

Los siguientes grandes contribuyentes a la masa son los Neutrófilos. Aunque son más pequeños que los macrófagos, su alto número total en la sangre y la médula ósea compensa su menor masa individual, dándoles un gran peso colectivo. Otros tipos como las células Dendríticas, los Linfocitos T y los Linfocitos B aportan masas significativas, pero son claramente menores en comparación con los dos líderes.

Esta distribución de masa es crítica para nuestro estudio. Si el aumento del Tejido Adiposo —nuestro modelo de obesidad— añade células inmunitarias, lo hará predominantemente a través de las células que ya residen allí en gran número.
- El TA es un sitio clave para los Macrófagos (Macrófagos asociados a la grasa). Por lo tanto, el aumento del TA en nuestro modelo previsiblemente potenciará aún más el dominio de los macrófagos en la carga inmunitaria total. La simulación cuantificará este efecto.

### Simulación: +20% tejido adiposo

In [99]:

adipose_row = df_mass_density[df_mass_density["tissue"].str.contains("Adipose", case=False)]
if adipose_row.empty:
    raise ValueError("No se encontró el tejido adiposo. Revisa el nombre exacto en la hoja 'Tissues'.")

adipose_mass_ref_g = float(adipose_row["tissue_mass_g"].iloc[0])

adipose_cell_dens = cell_type_densities[
    cell_type_densities["tissue"].str.contains("Adipose", case=False)
][["cell_type", "density"]].copy()

adipose_numbers_base = adipose_cell_dens.assign(
    number=lambda d: d["density"] * adipose_mass_ref_g
)
adipose_mass_new_g = adipose_mass_ref_g * 1.20
adipose_numbers_new = adipose_cell_dens.assign(
    number=lambda d: d["density"] * adipose_mass_new_g
)

delta_numbers = (
    adipose_numbers_new.set_index("cell_type")["number"]
    - adipose_numbers_base.set_index("cell_type")["number"]
)

delta_cells_total = float(delta_numbers.sum())
delta_mass_total_g = float(
    (delta_numbers * avg_mass_per_cell.reindex(delta_numbers.index)).sum()
)

print(f"Células adicionales por +20% adiposo: {delta_cells_total:,.0f}")
print(f"Masa inmunitaria adicional estimada: {delta_mass_total_g:,.4f} g")


Células adicionales por +20% adiposo: 1,135,272,491
Masa inmunitaria adicional estimada: 2.3801 g


**Interpretación del escenario:** al aumentar la masa del tejido adiposo manteniendo su densidad, todo el contenido inmunitario de ese tejido crece de forma proporcional. Esto imita una obesidad geométrica, más volumen, mismo tejido. El resultado esperado es un aumento absoluto de células inmunes, pero tendremos que comparar ese aumento con el total del organismo para saber si realmente es relevante.

Hemos establecido que el Tejido Adiposo (TA) es un órgano de baja densidad pero potencialmente alta masa. Ahora, aplicamos nuestro modelo de obesidad simple: un aumento del 20% en la masa del tejido adiposo, manteniendo su densidad original. Esto simula un crecimiento geométrico de la grasa.

Para cuantificar el impacto, hicimos lo siguiente:
- Tomamos la masa de referencia del tejido adiposo y la multiplicamos por 1.20.
- Mantenemos la densidad celular constante (células/gramo).
- Calculamos el número de células adicionales y la masa adicional que esta nueva grasa aporta al sistema inmunitario total."

Los números son significativos en términos absolutos:
- Células Adicionales: El aumento del 20% en el TA suma 1,135,272,491 células inmunitarias extra. ¡Más de un billón de células!
- Masa Inmunitaria Adicional: Esto se traduce en una Masa Inmunitaria adicional de 2.38 gramos.

El desafío, sin embargo, no está en el número absoluto, sino en el contexto. Un billón de células parece enorme, pero debemos compararlo con la línea base que establecimos al inicio del informe:
- Línea Base Total de Células: $1,837,950,387,444$ células (casi 1.84 billones).
La pregunta crucial ahora es: ¿Qué tan relevante es este billón adicional de células cuando lo comparamos con el total de 1.84 billones que ya teníamos? ¿Cambia fundamentalmente el 'peso' del sistema inmunitario, o el efecto es local y pequeño en el gran esquema?


In [100]:

compare_df = adipose_numbers_base.merge(
    adipose_numbers_new, on="cell_type", suffixes=("_ref", "_new")
)

fig5 = go.Figure(data=[
    go.Bar(name="Escenario basal", x=compare_df["cell_type"], y=compare_df["number_ref"]),
    go.Bar(name="+20% adiposo", x=compare_df["cell_type"], y=compare_df["number_new"])
])
fig5.update_layout(
    barmode="group",
    title="Gráfica 5. Células inmunes en tejido adiposo: antes vs después",
    xaxis_title="Tipo celular",
    yaxis_title="Células estimadas"
)
mostrar(fig5)


**Gráfica 5.** Todas las barras del escenario “+20% adiposo” están por encima de las basales porque el modelo es proporcional. La utilidad de esta figura es que permite ver **qué tipos celulares había realmente en el tejido adiposo**: los que tienen barras más grandes son los que más contribuyen al aumento total. Esto se puede comentar en el informe como posible punto de inflamación local.

La simulación nos dijo que se añadió más de un billón de células en total. Pero, ¿qué tipo de células son estas? La Figura 5 nos responde. Aquí vemos el desglose de células inmunitarias dentro del Tejido Adiposo —y solo en el Tejido Adiposo— comparando el Escenario Basal (azul) con el +20% Adiposo (naranja).

Como era de esperar, dado que la simulación es geométrica, todas las barras naranjas son mayores que las azules. Esto es lógico: si el tejido crece un 20%, el contenido celular de cada tipo dentro de él también crece un 20%. La verdadera utilidad de esta gráfica es ver qué tipos celulares absorben el mayor aumento en términos absolutos.

El mensaje más fuerte es la confirmación de lo que ya anticipábamos en la Figura 4: el aumento es absorbido principalmente por los Macrófagos.
- La barra del macrófago es, por mucho, la más grande en ambos escenarios. Al ser el tipo celular más numeroso en el tejido adiposo, son ellos quienes contribuyen con la mayoría de ese billón de células adicionales. El incremento en el número de macrófagos es visualmente innegable.

Si bien hay aumentos en las Células T, Células B, y Células Plasmáticas, son marginales en comparación con el impacto en los Macrófagos.
- Esto es un punto crucial para el informe, ya que sugiere un posible punto de inflamación local. Los macrófagos en el tejido adiposo están fuertemente asociados con la inflamación de bajo grado característica de la obesidad. Un aumento tan grande y selectivo de la población de macrófagos es, por definición, un aumento del potencial inflamatorio dentro de la grasa.

Hemos visto el aumento local en el tejido adiposo. Ahora, debemos dar el paso final para responder a nuestra pregunta de investigación: ¿Hasta qué punto este aumento masivo en el TA modifica el reparto global del sistema inmunitario en todo el organismo? La siguiente diapositiva nos dará las cifras finales de cambio total.

In [101]:
kpi_df = pd.DataFrame({
    "Indicador": [
        "Células inmunitarias totales (referencia)",
        "Masa inmunitaria total (referencia)",
        "Células adicionales (+20% adiposo)",
        "Masa inmunitaria adicional (+20% adiposo)",
        "Aumento relativo de masa inmunitaria",
    ],
    "Valor": [
        f"{total_cells_ref:,.0f}",
        f"{total_mass_ref_g:,.2f} g",
        f"{delta_cells_total:,.0f}",
        f"{delta_mass_total_g:,.4f} g",
        f"{(delta_mass_total_g/total_mass_ref_g)*100:.5f} %",
    ]
})
display(kpi_df)


,Indicador,Valor
0,Células inmunitarias totales (referencia),"1,837,950,387,444"
1,Masa inmunitaria total (referencia),"1,224.49 g"
2,Células adicionales (+20% adiposo),"1,135,272,491"
3,Masa inmunitaria adicional (+20% adiposo),2.3801 g
4,Aumento relativo de masa inmunitaria,0.19438 %



**Tabla 1.** Hemos llegado al resumen cuantitativo de nuestro experimento. Esta tabla compara los valores iniciales de referencia con el impacto generado por el aumento del 20% en la masa del Tejido Adiposo.

Recordemos nuestra línea base: el individuo de referencia tiene 1.84 billones de células inmunitarias totales, con una masa de 1,224 gramos.
- La simulación, tal como vimos, añadió más de mil millones de células (1,135,272,491) y 2.38 gramos de masa inmunitaria adicional. En términos absolutos, el número es grande.

Pero la pregunta central era: ¿Hasta qué punto este cambio modifica el reparto cuantitativo de las células inmunitarias en el organismo?
- La respuesta está en la última fila: el Aumento Relativo de Masa Inmunitaria es de tan solo $0.19438\%$.
- Esto significa que, aunque el tejido adiposo ha crecido un 20% y ha añadido más de un billón de células a su masa, el impacto en el sistema inmunitario global del organismo es mínimo, apenas una quinta parte de un punto porcentual.

Nuestro análisis unificado de las cuatro gráficas nos lleva a la siguiente conclusión clave:
- El sistema inmunitario global es extremadamente estable y no es 'desbordado' por cambios en tejidos periféricos de baja densidad. Los centros de alta densidad (Bazo, Ganglios) mantienen su peso relativo sin cambios.
- El efecto del aumento del TA es primariamente local: concentra más Macrófagos en la grasa (Figura 5), elevando el potencial de inflamación local asociada a la obesidad.
- El aumento de la masa del tejido adiposo no modifica significativamente el reparto cuantitativo del sistema inmunitario a nivel global, pero sí crea un foco de cambio cualitativo y local.


## 3. Conclusiones

1. La distribución inmunitaria del cuerpo es **heterogénea**: algunos tejidos pequeños son muy activos y otros grandes apenas contribuyen en densidad.
2. El tejido adiposo no es de los más densos, pero su **crecimiento** implica automáticamente más células inmunes residiendo allí.
3. El efecto **global** sobre la masa inmunitaria total es pequeño: el organismo mantiene una configuración estable incluso con más grasa.
4. El modelo usado es proporcional; en obesidad real podrían aumentar de forma preferente ciertos tipos celulares (macrófagos proinflamatorios).
5. Las visualizaciones interactivas permiten a quien corrija el trabajo explorar los datos, lo que aporta valor añadido frente a un informe estático.

## 4. Limitaciones

- Modelo basado en un único individuo de referencia (73 kg, masculino).
- Se asume densidad constante al aumentar la masa de un tejido.
- No se modelan respuestas sistémicas ni redistribución.



In [103]:
from pathlib import Path
from datetime import datetime

figuras = [
    (fig1, "Figura 1. Tejidos con mayor densidad de células inmunitarias",
     "Los tejidos linfoides dominan por su alta densidad celular, mostrando su papel esencial en la defensa inmune."),
    (fig2, "Figura 2. Contribución de cada tejido a la carga inmunitaria total",
     "El treemap revela que tejidos grandes pero poco densos, como la sangre, aportan significativamente al total corporal."),
    (fig3, "Figura 3. Masa del tejido vs densidad inmunitaria (log-log)",
     "No existe una relación lineal: algunos órganos pequeños concentran más células inmunes que tejidos grandes."),
    (fig4, "Figura 4. Distribución de la masa inmunitaria por tipo celular",
     "Los macrófagos y neutrófilos destacan por su gran contribución en masa, aunque no siempre en número."),
    (fig5, "Figura 5. Células inmunes en tejido adiposo antes y después del +20%",
     "El aumento del tejido adiposo incrementa proporcionalmente las células inmunes en ese tejido."),
]

css_styles = """
<style>
    * {
        margin: 0;
        padding: 0;
        box-sizing: border-box;
    }
    
    :root {
        --bg-primary: #0f1419;
        --bg-secondary: #1a1f2e;
        --bg-tertiary: #252d3d;
        --text-primary: #e8eef5;
        --text-secondary: #b4bcc8;
        --accent-primary: #00d9ff;
        --accent-secondary: #0ea5e9;
        --border-color: #2a3447;
        --shadow-sm: 0 4px 12px rgba(0, 0, 0, 0.4);
        --shadow-md: 0 8px 24px rgba(0, 0, 0, 0.5);
        --shadow-lg: 0 16px 40px rgba(0, 0, 0, 0.6);
    }
    
    html {
        scroll-behavior: smooth;
    }
    
    body {
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', sans-serif;
        background: linear-gradient(135deg, var(--bg-primary) 0%, var(--bg-secondary) 100%);
        color: var(--text-primary);
        line-height: 1.6;
        overflow-x: hidden;
    }
    
    /* Header elegante */
    .header {
        background: linear-gradient(135deg, rgba(15, 20, 25, 0.9) 0%, rgba(10, 40, 60, 0.9) 100%);
        backdrop-filter: blur(20px);
        border-bottom: 1px solid var(--border-color);
        padding: 4rem 2rem;
        text-align: center;
        position: relative;
        overflow: hidden;
    }
    
    .header::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background: radial-gradient(circle at 30% 50%, rgba(0, 217, 255, 0.05) 0%, transparent 50%);
        pointer-events: none;
    }
    
    .header-content {
        position: relative;
        z-index: 1;
    }
    
    .header h1 {
        font-size: 2.8rem;
        font-weight: 700;
        margin-bottom: 0.8rem;
        letter-spacing: -1px;
        background: linear-gradient(135deg, var(--accent-primary) 0%, var(--accent-secondary) 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
    }
    
    .header p {
        font-size: 1.05rem;
        color: var(--text-secondary);
        max-width: 650px;
        margin: 0 auto;
        font-weight: 300;
        letter-spacing: 0.5px;
    }
    
    .header-accent {
        width: 60px;
        height: 3px;
        background: linear-gradient(90deg, var(--accent-primary), var(--accent-secondary));
        margin: 1.5rem auto;
        border-radius: 2px;
    }
    
    /* Contenedor principal */
    .container {
        max-width: 1050px;
        margin: 0 auto;
        padding: 0 1.5rem;
    }
    
    /* Sección de figuras */
    .figures-section {
        padding: 3rem 0;
    }
    
    /* Tarjeta para cada figura */
    .figure-card {
        background: linear-gradient(135deg, var(--bg-secondary) 0%, rgba(25, 33, 46, 0.7) 100%);
        border: 1px solid var(--border-color);
        border-radius: 16px;
        padding: 2.8rem;
        margin-bottom: 2.5rem;
        box-shadow: var(--shadow-sm);
        transition: all 0.4s cubic-bezier(0.23, 1, 0.320, 1);
        position: relative;
        overflow: hidden;
    }
    
    .figure-card::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        height: 1px;
        background: linear-gradient(90deg, transparent, var(--accent-primary), transparent);
        opacity: 0.3;
    }
    
    .figure-card:hover {
        transform: translateY(-4px);
        box-shadow: var(--shadow-lg);
        border-color: var(--accent-primary);
        background: linear-gradient(135deg, rgba(25, 33, 46, 0.9) 0%, rgba(10, 40, 60, 0.5) 100%);
    }
    
    .figure-card::after {
        content: '';
        position: absolute;
        top: 0;
        left: -100%;
        width: 100%;
        height: 100%;
        background: linear-gradient(90deg, transparent, rgba(0, 217, 255, 0.1), transparent);
        transition: left 0.6s ease;
        pointer-events: none;
    }
    
    .figure-card:hover::after {
        left: 100%;
    }
    
    .figure-card-header {
        margin-bottom: 1.8rem;
        padding-bottom: 1.2rem;
        border-bottom: 1px solid var(--border-color);
    }
    
    .figure-card h2 {
        color: var(--accent-primary);
        font-size: 1.6rem;
        font-weight: 600;
        margin: 0;
        letter-spacing: -0.5px;
    }
    
    .figure-number {
        font-size: 0.85rem;
        color: var(--text-secondary);
        font-weight: 500;
        margin-bottom: 0.5rem;
    }
    
    /* Contenedor del gráfico */
    .chart-container {
        margin: 1.8rem 0;
        padding: 1.5rem;
        background: rgba(10, 20, 30, 0.6);
        border: 1px solid var(--border-color);
        border-radius: 12px;
        overflow: hidden;
        position: relative;
    }
    
    .chart-container::before {
        content: '';
        position: absolute;
        inset: 0;
        background: linear-gradient(135deg, rgba(0, 217, 255, 0.02) 0%, rgba(14, 165, 233, 0.02) 100%);
        pointer-events: none;
    }
    
    .chart-container > div {
        width: 100%;
        position: relative;
        z-index: 1;
    }
    
    /* Texto de interpretación */
    .interpretation {
        margin-top: 1.8rem;
        padding: 1.5rem;
        background: linear-gradient(135deg, rgba(0, 217, 255, 0.08) 0%, rgba(14, 165, 233, 0.08) 100%);
        border: 1px solid rgba(0, 217, 255, 0.2);
        border-left: 3px solid var(--accent-primary);
        border-radius: 10px;
        font-size: 0.98rem;
        line-height: 1.8;
        color: var(--text-secondary);
    }
    
    .interpretation strong {
        color: var(--accent-primary);
        font-weight: 600;
    }
    
    .interpretation-icon {
        display: inline-block;
        margin-right: 0.5rem;
        font-size: 1.2rem;
    }
    
    /* Footer elegante */
    .footer {
        text-align: center;
        padding: 3rem 2rem;
        border-top: 1px solid var(--border-color);
        margin-top: 4rem;
        background: linear-gradient(180deg, transparent 0%, rgba(10, 40, 60, 0.3) 100%);
    }
    
    .footer p {
        color: var(--text-secondary);
        font-size: 0.9rem;
        margin: 0.5rem 0;
        font-weight: 300;
    }
    
    .footer-accent {
        display: inline-block;
        width: 40px;
        height: 2px;
        background: linear-gradient(90deg, var(--accent-primary), var(--accent-secondary));
        margin: 1rem 0;
        border-radius: 1px;
    }
    
    .footer-timestamp {
        color: var(--text-secondary);
        font-size: 0.85rem;
        margin-top: 1rem;
    }
    
    /* Estilos responsivos */
    @media (max-width: 768px) {
        .header h1 {
            font-size: 2rem;
        }
        
        .header p {
            font-size: 0.95rem;
        }
        
        .figure-card {
            padding: 1.8rem;
        }
        
        .figure-card h2 {
            font-size: 1.3rem;
        }
        
        .header {
            padding: 2.5rem 1.5rem;
        }
        
        .container {
            padding: 0 1rem;
        }
    }
    
    /* Animación de carga suave */
    @keyframes fadeInUp {
        from {
            opacity: 0;
            transform: translateY(20px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }
    
    .figure-card {
        animation: fadeInUp 0.6s ease-out forwards;
    }
    
    .figure-card:nth-child(1) { animation-delay: 0.1s; }
    .figure-card:nth-child(2) { animation-delay: 0.2s; }
    .figure-card:nth-child(3) { animation-delay: 0.3s; }
    .figure-card:nth-child(4) { animation-delay: 0.4s; }
    .figure-card:nth-child(5) { animation-delay: 0.5s; }
    
    /* Scrollbar personalizada */
    ::-webkit-scrollbar {
        width: 10px;
    }
    
    ::-webkit-scrollbar-track {
        background: var(--bg-primary);
    }
    
    ::-webkit-scrollbar-thumb {
        background: var(--border-color);
        border-radius: 5px;
    }
    
    ::-webkit-scrollbar-thumb:hover {
        background: var(--accent-secondary);
    }
</style>
"""

html_parts = []
html_parts.append("""<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Análisis del Sistema Inmunitario | Informe Bioinformático Profesional</title>
    <meta name="description" content="Análisis avanzado e interactivo de la distribución de células inmunitarias en el cuerpo humano">
""")

html_parts.append(css_styles)

html_parts.append("""
</head>
<body>
    <div class="header">
        <div class="header-content">
            <h1>Análisis del Sistema Inmunitario</h1>
            <div class="header-accent"></div>
            <p>Visualización interactiva y análisis avanzado de la densidad y distribución de células inmunitarias corporales</p>
        </div>
    </div>
    
    <div class="container">
        <div class="figures-section">
""")

for idx, (fig, titulo, descripcion) in enumerate(figuras, 1):
    html_parts.append(f"""
        <div class="figure-card">
            <div class="figure-card-header">
                <div class="figure-number">Análisis {idx} de 5</div>
                <h2>{titulo.split('. ')[1]}</h2>
            </div>
            <div class="chart-container">
""")
    html_parts.append(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    html_parts.append(f"""
            </div>
            <div class="interpretation">
                <strong>Interpretación:</strong> {descripcion}
            </div>
        </div>
""")

timestamp = datetime.now().strftime("%d de %B de %Y a las %H:%M")

html_parts.append(f"""
        </div>
    </div>
    
    <div class="footer">
        <div class="footer-accent"></div>
        <p>Informe Generado Automáticamente</p>
        <p style="font-weight: 600; color: var(--accent-primary); margin-top: 1rem;">Análisis Bioinformático de Inmunidad</p>
        <div class="footer-timestamp">
            <p>Última actualización: {timestamp}</p>
        </div>
    </div>
</body>
</html>
""")

# Guardar HTML
html_path = Path("Informe_Bioinformatica_Inmunidad.html")
with open(html_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html_parts))

print(f"Ubicación: {html_path.resolve()}")

Ubicación: C:\Users\Adrian Ojeda\Desktop\ulpgc\CUARTO_CURSO\BIOINFORMÁTICA\BIOINFORMATICA-ANALISIS-SIH\Informe_Bioinformatica_Inmunidad.html
